<a href="https://colab.research.google.com/github/Enell261/Capstone_project/blob/data/Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**INTRODUCTION**

Pretoria East is the most vibrant district in the city of Pretoria, South Africa. It boasts a large student population owing to the fact that there are 3 universities within the district and numerous colleges. It is also home to a majority of the most affluent neighborhoods in the city. Accommodation for students that provides a safe and vibrant environment whilst offering access to public transport is notoriously scarce in the city.<br>
New students often end up living in dangerous neighbourhoods that are far from campuses and do not offer places that enhance student life.<br>
This project aims to cluster the neighborhoods in Pretoria East to help students find the ones that best serve their needs. The project will make use of the Foursquare API to obtain data on the various neighborhoods in the district. Specifically, the project will cluster student-friendly neighbourhoods together so as to suggest these to students seeking accomodation

**DATA**

The data used in the project includes a Wikipedia list of all surbubs in Pretoria East and Foursquare data corresponding to each of the surbubs in the list. The Foursquare data will be procured using their API for which we have a developer account. The data will come in the standard Foursquare format